In [1]:
#from boson_data_lib import *
import numpy as np
import scipy.linalg as sl
import os
import pandas as pd
import glob
import h5py

In [2]:
data_dir = "../DATA/"
tests_dir = "" #"../TESTS/"

In [3]:
#best_init_rho =  np.array([[ 0.96693975+0.j        , -0.17832301+0.00560344j],
#                               [-0.17832301-0.00560344j,  0.03306025+0.j        ]])#

#fid_dists = fidelity_distances(data_dir, best_init_rho);

def extract_time(f_name, g):
    with h5py.File(f_name, 'r') as f:

        t = np.array(f[g]['t']).reshape(-1)
        t = t.flatten()
        dt = t[1] - t[0]
        assert ((np.abs((t[1:] - t[:-1]) - dt) < 1e-6).all())

    return t, dt

In [4]:
def read_as_dataframe(filename, data_dir, tests_dir): #fid_distances,

    df = pd.DataFrame()
    
    print("Processing...")
    
    with h5py.File(tests_dir+filename, "r") as f:
        
        #gammas = ['0.079477', '0.25133', '0.79477', '2.5133', '7.9477', '25.133', '79.477', '251.33']
        gammas = f.keys()
        
        for gamma in gammas:
            
            print(gamma)
            
            g = gamma[6:]
            
            init_states = ["D"+str(i) for i in range(1, 11)] #f[gamma].keys()
            
            for state in init_states:
                
                Fidelity = f[gamma][state]["Fidelity"][()]

                ser_len = len(Fidelity)

                gamma_column = [g] * ser_len
                state_column = [state[7:]] * ser_len

                #fid_dist_column = [fid_distances[int(state[7:])-1]] * ser_len      

                f_name = data_dir + "/State_" + state + "_2CUT_data.h5"
                #f_name = data_dir + "\\" + state + "_2CUT_data.h5"
                t, dt = extract_time(f_name, g)
                #time_column = t
                time_column = np.arange(t[0], t[0]+dt*ser_len, dt, dtype=float)          

                gamma_time_column = np.array(time_column)*float(g) 

                d_ser = {'Gamma': gamma_column,
                         'State': state_column,
                         'Time': time_column, 
                         'gt': gamma_time_column,
                         'Fidelity': Fidelity,
                         'Infidelity': 1-Fidelity} #,
                         #'Distance': fid_dist_column}

                df_ser = pd.DataFrame(data = d_ser)   
                df = pd.concat([df, df_ser])
    
    print(" done!")
    
    return df

In [5]:
#test_file = "KOSSAK_UNCONSTR_BFGS1_trn4_tst20_2024-Jul-30_at_08-26.h5"
test_file = "KOSSAK_UNCONSTR_RK_BFGS1_trn4_tst20_2024-Aug-01_at_03-28.h5"

In [6]:
df = read_as_dataframe(test_file, data_dir, tests_dir) #fid_dists,

Processing...
gamma_0.079477
gamma_0.25133
gamma_0.79477
gamma_2.5133
gamma_25.133
gamma_251.33
gamma_7.9477
gamma_79.477
 done!


In [7]:
df

,Gamma,State,Time,gt,Fidelity,Infidelity
0,0.079477,,0.06000,0.004769,1.000000,3.219647e-15
1,0.079477,,0.08000,0.006358,0.999998,1.870530e-06
2,0.079477,,0.10000,0.007948,0.999989,1.097106e-05
3,0.079477,,0.12000,0.009537,0.999984,1.559943e-05
4,0.079477,,0.14000,0.011127,0.999994,5.828959e-06
...,...,...,...,...,...,...
1996,79.477,,0.54925,43.652742,0.994531,5.469340e-03
1997,79.477,,0.54950,43.672612,0.994534,5.466477e-03
1998,79.477,,0.54975,43.692481,0.994536,5.463608e-03
1999,79.477,,0.55000,43.712350,0.994539,5.460734e-03


In [8]:
pkl_name = "dataframe_"+test_file[:-3]+".pkl"

df.to_pickle(tests_dir+pkl_name)

In [9]:
pkl_name

'dataframe_KOSSAK_UNCONSTR_RK_BFGS1_trn4_tst20_2024-Aug-01_at_03-28.pkl'

In [43]:
tests_dir+pkl_name

'dataframe_KOSSAK_UNCONSTR_BFGS1_trn4_tst20_2024-Jul-30_at_08-26.pkl'